In [1]:
from pathlib import Path
import pandas as pd
from prefect import flow, task
from prefect_gcp.cloud_storage import GcsBucket
from random import randint
from prefect_gcp import GcpCredentials

In [2]:
@task(retries=3)
def fetch(dataset_url: str) -> pd.DataFrame:
    """Read taxi data from web into pandas DataFrame"""
    # if randint(0, 1) > 0:
    #     raise Exception

    df = pd.read_csv(dataset_url)
    return df

@task(log_prints=True)
def clean(df: pd.DataFrame, color: str) -> pd.DataFrame:
    """Fix dtype issues"""

    if color == "yellow":
        df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
        df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
    elif(color == "green"):
        df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])
        df["lpep_dropoff_datetime"] = pd.to_datetime(df["lpep_dropoff_datetime"])
    elif(color == "fhv"):
        df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
        df["dropOff_datetime"] = pd.to_datetime(df["dropOff_datetime"])

    print(df.head(2))
    print(f"columns: {df.dtypes}")
    print(f"rows: {len(df)}")
    return df

@task()
def write_local(df: pd.DataFrame, color: str, dataset_file: str) -> Path:
    """Write DataFrame out locally as parquet file"""
    path = Path(f"data/{color}/{dataset_file}.parquet")
    df.to_parquet(path, compression="gzip")
    return path

@task()
def write_gcs(path: Path) -> None:
    """Upload local parquet file to GCS"""
    gcs_block = GcsBucket.load("zoom-gcs")
    gcs_block.upload_from_path(from_path=path, to_path=path)
    return

@flow()
def etl_web_to_gcs() -> None:
    """The main ETL function"""
    colors = ["yellow","green", "fhv"]
    year = [2019,2020]
    month = list(range(1,13))

    for color in colors:
        for j in year:
            for i in month:
                dataset_file = f"{color}_tripdata_{j}-{i:02}"
                dataset_url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/{color}/{dataset_file}.csv.gz"
                df = fetch(dataset_url)
                df_clean = clean(df, color)
                path = write_local(df_clean, color, dataset_file)
                write_gcs(path)

if __name__ == "__main__":
    etl_web_to_gcs()

10:29:03.343 | INFO    | prefect.engine - Created flow run 'glossy-dragon' for flow 'etl-web-to-gcs'

10:29:03.418 | INFO    | Flow run 'glossy-dragon' - Created task run 'fetch-b4598a4a-0' for task 'fetch'

10:29:03.419 | INFO    | Flow run 'glossy-dragon' - Executing 'fetch-b4598a4a-0' immediately...

10:29:35.382 | INFO    | Task run 'fetch-b4598a4a-0' - Finished in state Completed()

10:29:35.402 | INFO    | Flow run 'glossy-dragon' - Created task run 'clean-b9fd7e03-0' for task 'clean'

...

10:52:29.980 | ERROR   | Task run 'clean-b9fd7e03-60' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3803, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'dropOff_datetime'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/engine.py", line 1445, in orchestrate_task_run
    result = await run_sync(task.fn, *args, **kwargs)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_79483/3077171401.py", line 22, in clean
    df["dropOff_datetime"] = pd.to_datetime(df["dropOff_datetime"])
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/pandas/core/frame.py", line 3805, in __getitem__
    indexer = self.columns.get_loc(key)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    raise KeyError(key) from err
KeyError: 'dropOff_datetime'

10:52:29.999 | ERROR   | Task run 'clean-b9fd7e03-60' - Finished in state Failed('Task run encountered an exception: Traceback (most recent call last):\n  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3803, in get_loc\n    return self._engine.get_loc(casted_key)\n  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc\n  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc\n  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item\n  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item\nKeyError: \'dropOff_datetime\'\n\nThe above exception was the direct cause of the following exception:\n\nKeyError: \'dropOff_datetime\'\n')

10:52:30.001 | ERROR   | Flow run 'glossy-dragon' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3803, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'dropOff_datetime'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/engine.py", line 636, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_79483/3077171401.py", line 56, in etl_web_to_gcs
    df_clean = clean(df, color)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/tasks.py", line 436, in __call__
    return enter_task_run_engine(
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/engine.py", line 926, in enter_task_run_engine
    return run_async_from_worker_thread(begin_run)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 177, in run_async_from_worker_thread
    return anyio.from_thread.run(call)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/from_thread.py", line 49, in run
    return asynclib.run_async_from_thread(func, *args)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 970, in run_async_from_thread
    return f.result()
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/concurrent/futures/_base.py", line 446, in result
    return self.__get_result()
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/concurrent/futures/_base.py", line 391, in __get_result
    raise self._exception
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/engine.py", line 1064, in get_task_call_return_value
    return await future._result()
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/futures.py", line 237, in _result
    return await final_state.result(raise_on_failure=raise_on_failure, fetch=True)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/states.py", line 89, in _get_state_result
    raise await get_state_exception(state)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/engine.py", line 1445, in orchestrate_task_run
    result = await ru

10:52:30.029 | ERROR   | Flow run 'glossy-dragon' - Finished in state Failed('Flow run encountered an exception. Traceback (most recent call last):\n  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3803, in get_loc\n    return self._engine.get_loc(casted_key)\n  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc\n  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc\n  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item\n  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item\nKeyError: \'dropOff_datetime\'\n\nThe above exception was the direct cause of the following exception:\n\nKeyError: \'dropOff_datetime\'\n')

KeyError: 'dropOff_datetime'

In [2]:
print("Hello world")

Hello world


In [7]:
@task(retries=3)
def extract_from_gcs(color: str, year: int, month: int) -> Path:
    """Download trip data from GCS"""
    gcs_path = f"data/{color}/{color}_tripdata_{year}-{month:02}.parquet"
    # gcs_block = GcsBucket.load("zoom-gcs")
    # gcs_block.get_directory(from_path=gcs_path, local_path=f"./data/")
    return Path(f"./data/{gcs_path}")


@task()
def transform(path: Path) -> pd.DataFrame:
    """Data cleaning example"""
    df = pd.read_parquet(path)
    # print(f"pre: missing passenger count: {df['passenger_count'].isna().sum()}")
    # df["passenger_count"].fillna(0, inplace=True)
    # print(f"post: missing passenger count: {df['passenger_count'].isna().sum()}")
    return df


@task()
def write_bq(df: pd.DataFrame, color: str) -> None:
    """Write DataFrame to BiqQuery"""

    gcp_credentials_block = GcpCredentials.load("zoom-gcp-creds")

    destination_table = "trips_data_all."+color+"_tripdata"

    df.to_gbq(
        destination_table=destination_table,
        project_id="esoteric-pen-376110",
        credentials=gcp_credentials_block.get_credentials_from_service_account(),
        chunksize=500_000,
        if_exists="append",
    )

@flow()
def etl_gcs_to_bq():
    """Main ETL flow to load data into Big Query"""

    colors = ["yellow","green", "fhv"]
    year = [2019,2020]
    month = list(range(1,13))

    for color in colors:
        for j in year:
            for i in month:
                # path = extract_from_gcs(color, i, j)

                if (color != "fhv") or ( year != "2020"):

                    dataset_file = f"{color}_tripdata_{j}-{i:02}"
                    
                    # df = pd.read_parquet(f"/data/{color}/{color}_tripdata_{i}-{j:02}.parquet")
                    df = pd.read_parquet(f"data/{color}/{dataset_file}.parquet")
                    write_bq(df, color)

if __name__ == "__main__":
    etl_gcs_to_bq()


/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/tasks.py:256: UserWarning: A task named 'extract_from_gcs' and defined at '/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_82817/3550712600.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/tasks.py:256: UserWarning: A task named 'transform' and defined at '/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_82817/3550712600.py:10' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/tasks.py:256: UserWarning: A task named 'write_bq' and defined at '/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_82817/3550712600.py:20' conflicts

11:15:29.653 | INFO    | prefect.engine - Created flow run 'splendid-dugong' for flow 'etl-gcs-to-bq'

11:15:30.374 | INFO    | Flow run 'splendid-dugong' - Created task run 'write_bq-b366772c-0' for task 'write_bq'

11:15:30.376 | INFO    | Flow run 'splendid-dugong' - Executing 'write_bq-b366772c-0' immediately...

11:16:10.617 | INFO    | Task run 'write_bq-b366772c-0' - Finished in state Completed()

...

11:32:00.126 | INFO    | Flow run 'splendid-dugong' - Executing 'write_bq-b366772c-59' immediately...

11:32:09.782 | INFO    | Task run 'write_bq-b366772c-59' - Finished in state Completed()

11:32:09.784 | ERROR   | Flow run 'splendid-dugong' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/engine.py", line 636, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_82817/3550712600.py", line 54, in etl_gcs_to_bq
    df = pd.read_parquet(f"data/{color}/{dataset_file}.parquet")
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/pandas/io/parquet.py", line 503, in read_parquet
    return impl.read(
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/pandas/io/parquet.py", line 244, in read
    path_or_handle, handles, kwargs["filesystem"] = _get_path_or_handle(
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/pandas/io/parquet.py", line 102, in _get_path_or_handle
    handles = get_handle(
  File "/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/pandas/io/common.py", line 865, in get_handle
    handle = open(handle, ioargs.mode)
FileNotFoundError: [Errno 2] No such file or directory: 'data/fhv/fhv_tripdata_2020-01.parquet'

11:32:09.809 | ERROR   | Flow run 'splendid-dugong' - Finished in state Failed("Flow run encountered an exception. FileNotFoundError: [Errno 2] No such file or directory: 'data/fhv/fhv_tripdata_2020-01.parquet'\n")

FileNotFoundError: [Errno 2] No such file or directory: 'data/fhv/fhv_tripdata_2020-01.parquet'

## Week 4 Homework 

In this homework, we'll use the models developed during the week 4 videos and enhance the already presented dbt project using the already loaded Taxi data for fhv vehicles for year 2019 in our DWH.

We will use the data loaded for:
* Building a source table: stg_fhv_tripdata
* Building a fact table: fact_fhv_trips
* Create a dashboard 

If you don't have access to GCP, you can do this locally using the ingested data from your Postgres database
instead. If you have access to GCP, you don't need to do it for local Postgres -
only if you want to.

### Question 1: 
**What is the count of records in the model fact_trips after running all models with the test run variable disabled and filtering for 2019 and 2020 data only (pickup datetime)**  
You'll need to have completed the "Build the first dbt models" video and have been able to run the models via the CLI. 
You should find the views and models for querying in your DWH.



To disable the test run variable we can change the default value to false in -> staging/stg_green_tripdata.sql and staging/stg_yellow_tripdata.sql

```
-- dbt build --m <model.sql> --var 'is_test_run: false'
{% if var('is_test_run', default=false) %}

  limit 100

{% endif %}
```

![Alt text](images/img1.png "Cron")
![Alt text](images/img2.png "Cron")

### Question 2: 
**What is the distribution between service type filtering by years 2019 and 2020 data as done in the videos**

You will need to complete "Visualising the data" videos, either using data studio or metabase. 


![Alt text](images/img3.png "Cron")

### Question 3: 
**What is the count of records in the model stg_fhv_tripdata after running all models with the test run variable disabled (:false)**  

Create a staging model for the fhv data for 2019 and do not add a deduplication step. Run it via the CLI without limits (is_test_run: false).
Filter records with pickup time in year 2019.

![Alt text](images/img4.png "Cron")

### Question 4: 
**What is the count of records in the model fact_fhv_trips after running all dependencies with the test run variable disabled (:false)**  

Create a core model for the stg_fhv_tripdata joining with dim_zones.
Similar to what we've done in fact_trips, keep only records with known pickup and dropoff locations entries for pickup and dropoff locations. 
Run it via the CLI without limits (is_test_run: false) and filter records with pickup time in year 2019.

```
{{ config(materialized='view') }}

select * from {{ source('staging','fhv_tripdata') }}

-- dbt build --m <model.sql> --var 'is_test_run: false'
{% if var('is_test_run', default=false) %}

  limit 100

{% endif %}
--------------------------------------------------------
{{ config(materialized='table') }}

with dim_zones as (
    select * from {{ ref('dim_zones') }}
    where borough != 'Unknown'
),

fhv_data as (
    select *, 
        'Fhv' as fhv_type
    from {{ ref('stg_fhv_tripdata') }}
)

select 
    fhv_data.dispatching_base_num, 
    fhv_data.pickup_datetime, 
    fhv_data.dropOff_datetime, 
    fhv_data.PUlocationID,
    fhv_data.DOlocationID,
    fhv_data.SR_Flag,
    fhv_data.Affiliated_base_number,
    fhv_data.fhv_type
    
from fhv_data
inner join dim_zones as pickup_zone
on fhv_data.PUlocationID = pickup_zone.locationid
inner join dim_zones as dropoff_zone
on fhv_data.DOlocationID = dropoff_zone.locationid

```

![Alt text](images/img5.png "Cron")

### Question 5: 
**What is the month with the biggest amount of rides after building a tile for the fact_fhv_trips table**
Create a dashboard with some tiles that you find interesting to explore the data. One tile should show the amount of trips per month, as done in the videos for fact_trips, based on the fact_fhv_trips table.

January is the month with biggest amount of rides after building a tile.

![Alt text](images/im6.png "Cron")

[Link to report](https://lookerstudio.google.com/s/sKSRNwljHeo) 